In [1]:
import cv2
import os

from slide_detector_kenta_eval import slide_detector_kenta
from slide_detector_eval import slide_detector

In [2]:
PATH_TO_EXTRACTED = './eval/extracted'
EXTRACTED_FOLDERS = ['slide-detector', 'slide-detector-kenta']
EXTRACTED_VIDEOS = ['basdat_4', 'basdat_10', 'jarkom_6', 'jarkom_8', 'jarkom_9']

PATH_TO_GT = './eval/annotations/'
GT_FILE_TEMPLATE = '%s_gold_standard.txt'


def gt_file_name(video_name):
    return GT_FILE_TEMPLATE % video_name


def folder_path_to_file_names(path):
    return os.listdir(path)


def file_name_to_timestamp(file_name: str):
    split = file_name.split('-')
    m = int(split[0])
    s = int(split[1])
    ms = int(split[2].removesuffix('.jpg'))

    if ms > 0:
        s += 1

    if s >= 60:
        m += (s//60)
        s %= 60

    return '%02d-%02d' % (m, s)


def file_to_timestamps(file_path: str):
    with open(file_path, 'r') as file:
        lines = [*filter(lambda c: c != "", [line.replace("\n", "").replace(" ", "") for line in file.readlines()])]
    return lines

In [3]:
truth = file_to_timestamps(PATH_TO_GT + gt_file_name('basdat_4'))
pred_kenta = [*map(file_name_to_timestamp, folder_path_to_file_names(PATH_TO_EXTRACTED + '/slide-detector-kenta/' + 'basdat_4'))]
pred_raw = [*map(file_name_to_timestamp, folder_path_to_file_names(PATH_TO_EXTRACTED + '/slide-detector/' + 'basdat_4'))]

In [4]:
from sklearn.metrics import precision_score, recall_score, fbeta_score
import numpy as np

In [5]:
def iou(l1, l2):
    intersection = len(set(l1).intersection(set(l2)))
    union = len(set(l1).union(set(l2)))
    return intersection / union

print(iou(truth, pred_kenta))
print(iou(truth, pred_raw))

0.3617021276595745
0.37777777777777777


In [6]:
def calculate_precision_recall_f2score(ground_truth, prediction):
    true_positives = len(set(prediction).intersection(ground_truth))
    false_positives = len(prediction) - true_positives
    false_negatives = len(ground_truth) - true_positives

    # print(true_positives)
    # print(false_negatives)
    # print(false_positives)

    divisor = true_positives + false_positives
    precision = 0
    if divisor != 0:
        precision = true_positives / divisor

    divisor = true_positives + false_negatives
    recall = 0
    if divisor != 0:
        recall = true_positives / divisor

    if precision != 0 or recall != 0:
        beta = 2
        f2_score = ((1 + beta**2) * precision * recall) / ((beta**2 * precision) + recall)
    else:
        f2_score = 0

    return true_positives, false_positives, false_negatives, precision, recall, f2_score

print(calculate_precision_recall_f2score(truth, pred_kenta))
print(calculate_precision_recall_f2score(truth, pred_raw))

(34, 60, 0, 0.3617021276595745, 1.0, 0.7391304347826088)
(34, 56, 0, 0.37777777777777777, 1.0, 0.7522123893805309)


In [7]:
import pandas as pd
lst = []
for video in EXTRACTED_VIDEOS:
    truth = file_to_timestamps(PATH_TO_GT + gt_file_name(video))
    pred_kenta = [*map(file_name_to_timestamp, folder_path_to_file_names(PATH_TO_EXTRACTED + '/slide-detector-kenta/' + video))]
    pred_raw = [*map(file_name_to_timestamp, folder_path_to_file_names(PATH_TO_EXTRACTED + '/slide-detector/' + video))]
    iou_kenta = iou(truth, pred_kenta)
    iou_raw = iou(truth, pred_raw)
    [tp_kenta, fp_kenta, fn_kenta, precision_kenta, recall_kenta, f2_kenta] = calculate_precision_recall_f2score(truth, pred_kenta)
    [tp_raw, fp_raw, fn_raw, precision_raw, recall_raw, f2_raw] = calculate_precision_recall_f2score(truth, pred_raw)

    n_kenta = len(pred_kenta)
    n_raw = len(pred_raw)
    
    lst.append((video, iou_kenta, iou_raw, n_kenta, n_raw, tp_kenta, fp_kenta, fn_kenta, tp_raw, fp_raw, fn_raw, precision_kenta, precision_raw, recall_kenta, recall_raw, f2_kenta, f2_raw))

df = pd.DataFrame(lst, columns=["Video", "IOU-K", "IOU-R", "N-K", "N-R", "TP-K", "FP-K", "FN-K", "TP-R", "FP-R", "FN-R", "P-K", "P-R", "R-K", "R-R", "F2-K", "F2-R"])
df

,Video,IOU-K,IOU-R,N-K,N-R,TP-K,FP-K,FN-K,TP-R,FP-R,FN-R,P-K,P-R,R-K,R-R,F2-K,F2-R
0,basdat_4,0.361702,0.377778,94,90,34,60,0,34,56,0,0.361702,0.377778,1.000000,1.000000,0.739130,0.752212
1,basdat_10,0.941176,0.941176,16,16,16,0,1,16,0,1,1.000000,1.000000,0.941176,0.941176,0.952381,0.952381
2,jarkom_6,0.230769,0.000000,20,0,15,5,45,0,0,60,0.750000,0.000000,0.250000,0.000000,0.288462,0.000000
3,jarkom_8,0.220339,0.000000,21,0,13,8,38,0,0,51,0.619048,0.000000,0.254902,0.000000,0.288889,0.000000
4,jarkom_9,0.214286,0.000000,17,0,12,5,39,0,0,51,0.705882,0.000000,0.235294,0.000000,0.271493,0.000000


In [8]:
df.describe()

,IOU-K,IOU-R,N-K,N-R,TP-K,FP-K,FN-K,TP-R,FP-R,FN-R,P-K,P-R,R-K,R-R,F2-K,F2-R
count,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
mean,0.393655,0.263791,33.600000,21.200000,18.000000,15.600000,24.600000,10.000000,11.200000,32.600000,0.687326,0.275556,0.536275,0.388235,0.508071,0.340919
std,0.312067,0.412493,33.827504,39.079406,9.082951,24.985996,22.165288,15.099669,25.043961,29.534725,0.230613,0.436767,0.397083,0.532020,0.317426,0.472156
min,0.214286,0.000000,16.000000,0.000000,12.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.361702,0.000000,0.235294,0.000000,0.271493,0.000000
25%,0.220339,0.000000,17.000000,0.000000,13.000000,5.000000,1.000000,0.000000,0.000000,1.000000,0.619048,0.000000,0.250000,0.000000,0.288462,0.000000
50%,0.230769,0.000000,20.000000,0.000000,15.000000,5.000000,38.000000,0.000000,0.000000,51.000000,0.705882,0.000000,0.254902,0.000000,0.288889,0.000000
75%,0.361702,0.377778,21.000000,16.000000,16.000000,8.000000,39.000000,16.000000,0.000000,51.000000,0.750000,0.377778,0.941176,0.941176,0.739130,0.752212
max,0.941176,0.941176,94.000000,90.000000,34.000000,60.000000,45.000000,34.000000,56.000000,60.000000,1.000000,1.000000,1.000000,1.000000,0.952381,0.952381
